In [1]:
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import numpy as np
import json
import os 

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns=999
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
arquivosFatura = 'C:\\Temp\\ZDM_SD_SILO_ATTR.20220720.200957'

df = pd.read_csv(arquivosFatura
                ,sep=';'
                ,header=None
                ,error_bad_lines=False
                ,engine='python')

df[45].fillna('Y',inplace=True)

Skipping line 311056: Expected 46 fields in line 311056, saw 47


In [4]:
df.groupby([45]).agg(qtd=(45,'count')).reset_index()

,45,qtd
0,X,1733
1,Y,344126


In [17]:
arquivosSiloSAPBW = 'C:\\Temp\\ZDM_SD_SILO_ATTR.20220727.105251'

dfSiloSAPBW = pd.read_csv(arquivosSiloSAPBW
                ,sep=';'
                ,header=None
                ,error_bad_lines=False
                ,usecols=(0,1,2,45)
                ,names=['SOCNR','KUNNR','SEQNR','ITEM_DEL']
                ,dtype={'SOCNR':'str','KUNNR':'str','SEQNR':'str','ITEM_DEL':'str'}
                ,engine='python')

dfSiloSAPBW['ITEM_DEL'].fillna('Y',inplace=True)
dfSiloSAPBW.fillna('NA',inplace=True)

dfSiloSAPBW['ChaveSAPBW'] = dfSiloSAPBW['KUNNR']+ '_' + dfSiloSAPBW['SEQNR']+ '_' + dfSiloSAPBW['SOCNR']

In [6]:
dfSiloSAPBW.groupby(['ITEM_DEL']).agg(qtd=('ITEM_DEL','count')).reset_index()

,ITEM_DEL,qtd
0,00000000000000227,1
1,X,1966
2,Y,344360


In [22]:
dfSiloSAPBW

,SOCNR,KUNNR,SEQNR,ITEM_DEL,ChaveSAPBW
0,NA,NA,NA,X,NA_NA_NA
1,00000000000000000269,0001313400,PEDIDO,Y,0001313400_PEDIDO_00000000000000000269
2,00000000000000000270,0001316688,AUTOMATICO,Y,0001316688_AUTOMATICO_00000000000000000270
3,00000000000000000271,0001316504,AGENFAT,Y,0001316504_AGENFAT_00000000000000000271
4,00000000000000000272,0001109202,AGENDA,Y,0001109202_AGENDA_00000000000000000272
...,...,...,...,...,...
346322,00000000000000351003,0003946211,NA,Y,0003946211_NA_00000000000000351003
346323,00000000000000351004,0003857679,NA,Y,0003857679_NA_00000000000000351004
346324,00000000000000351005,0003949887,NA,Y,0003949887_NA_00000000000000351005
346325,00000000000000351006,0003548679,NA,Y,0003548679_NA_00000000000000351006


In [20]:
arquivosSiloSAP = 'C:\\Temp\\OIISOCIKN.csv'

dfSiloSAP = pd.read_csv(arquivosSiloSAP
                ,sep=';'
                ,header=0
                ,error_bad_lines=False
                ,usecols=(1,2,3,97)
                ,names=['KUNNR','SEQNR','SOCNR','ITEM_DEL']
                ,dtype={'KUNNR':'str','SEQNR':'str','SOCNR':'str','ITEM_DEL':'str'}                        
                ,engine='python')

dfSiloSAP['ITEM_DEL'].fillna('Y',inplace=True)

dfSiloSAP.fillna('NA',inplace=True)

dfSiloSAP['ChaveSAPECC'] = dfSiloSAP['KUNNR']+ '_' + dfSiloSAP['SEQNR']+ '_' + dfSiloSAP['SOCNR']

In [21]:
#dfSiloSAP[dfSiloSAP['SOCNR']=='00000000000000350339']

dfSiloSAP

,KUNNR,SEQNR,SOCNR,ITEM_DEL,ChaveSAPECC
0,0003907587,0000090035,00000000000000330799,Y,0003907587_0000090035_00000000000000330799
1,0003907610,0000090036,00000000000000330757,Y,0003907610_0000090036_00000000000000330757
2,0003907580,0000090036,00000000000000330758,Y,0003907580_0000090036_00000000000000330758
3,0003907582,0000090036,00000000000000330759,Y,0003907582_0000090036_00000000000000330759
4,0003907594,0000090035,00000000000000330801,Y,0003907594_0000090035_00000000000000330801
...,...,...,...,...,...
343877,0003938699,PADRÃO,00000000000000347087,Y,0003938699_PADRÃO_00000000000000347087
343878,0003939814,PADRAO,00000000000000347091,Y,0003939814_PADRAO_00000000000000347091
343879,0003938764,PRODUCAO,00000000000000347095,Y,0003938764_PRODUCAO_00000000000000347095
343880,0003939042,CENTRAL 1,00000000000000347111,Y,0003939042_CENTRAL 1_00000000000000347111


In [33]:
resultComparacaoSilo = pd.merge(left=dfSiloSAPBW
                      , right=dfSiloSAP
                      , left_on='SOCNR'
                      , right_on='SOCNR'
                      ,how='left')


In [40]:
resultComparacaoSilo['ChaveSAPECC'].fillna('Nao Encontrado SAP',inplace=True)

In [44]:
resultComparacaoSilo.rename(columns={'SOCNR':'CodSilo', 'KUNNR_x':'CodCliente', 'SEQNR_x':'CodOrdenacao', 'ITEM_DEL_x':'FlgSiloExcluido'}, inplace=True)

#resultComparacaoSilo[(resultComparacaoSilo['ChaveSAPBW'] != resultComparacaoSilo['ChaveSAPECC'])][['CodSilo','CodCliente','CodOrdenacao','FlgSiloExcluido']].to_csv('silo_validacao.csv', index=False, sep=';')

resultComparacaoSilo[(resultComparacaoSilo['ChaveSAPECC'] == 'Nao Encontrado SAP') & (resultComparacaoSilo['FlgSiloExcluido'] =='Y')][['CodSilo','CodCliente','CodOrdenacao','FlgSiloExcluido']].to_csv('silo_validacao.csv', index=False, sep=';')